In [1]:
from scipy.special import j1
from scipy.optimize import brentq
from sage.all import *
import struct

DR = RealField(52)

DD = RealField(157)

def double_to_hex(f):
    # Converts Python float (f64) to hex string
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)


In [4]:
from scipy.special import j1, jvp
from scipy.optimize import brentq
from sage.all import *
import struct

DR = RealField(52)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")
    
# Use Sage's RDF for high-precision floats
R120 = RealField(120)

# List of zeros
zeros = []

# Step size to detect sign changes
step = R120(0.001)
x = R120(0.0)

previous_zero = R120(0)
epsilon = 1e-12
j1_zeros = []

while x < 77:
    if j1(float(x)) * j1(float(x + step)) < 0:  # Sign change => zero in interval
        previous_zero = brentq(j1, x, x + step)
        j1_zeros.append(R120(previous_zero))
    if abs(x - round(x)) < epsilon:
        zeros.append(R120(previous_zero))
    x += step

j1_extrema = []

x = R120(0.0)
while x < 77:
    dfx = jvp(1, float(x), n=1)  # Derivative of J1
    dfx_next = jvp(1, float(x + step), n=1)
    if dfx * dfx_next < 0:
        extremum = brentq(lambda x: jvp(1, x, n=1), float(x), float(x + step))
        j1_extrema.append(R120(extremum))
    x += step

# Print results
for i, z in enumerate(j1_zeros):
    print(f"Zero {i+1}: x ≈ {z}")

print("Extrema (peaks/valleys) of J1(x):")
for e in j1_extrema:
    print(f"nExtrema: {e}")

j1_zeros.extend(j1_extrema)

j1_zeros = sorted(j1_zeros)

# Print results
for i, z in enumerate(j1_zeros):
    print(f"Peak or zero {i+1}: x ≈ {z}")

print("")

print("static J1_ZEROS: [(u64, u64); 46] = [")
for z in j1_zeros:
    k = split_double_double(DD(f"{z}"))
    hi = double_to_hex(k[1])
    lo = double_to_hex(k[0])
    print(f"({lo}, {hi}),")
    
print("];")

from mpmath import *
from sage.all import *

from mpmath import mp, j1, taylor, expm1

# Precision
mp.prec = 106

prev_zero = 0

CDR = RealField(107)

var('x')

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffs(poly, x0):
    print("TaylorExtendedSeries {")
    print_double_double("a0: ", poly[0])
    print_double_double("a1: ", poly[1])
    print_double_double("a2: ", poly[2])
    print_double_double("a3: ", poly[3])
    print_double_double("a4: ", poly[4])
    print_double_double("a5: ", poly[5])
    print_double_double("a6: ", poly[6])
    print("c: [")
    for i in range(7, 23):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")
    print("},")

mp.prec = 106

def print_expansion_at_0():
    print(f"pub(crate) static J1_MACLAURIN_SERIES: J1MaclaurinSeries = J1MaclaurinSeries {{")
    from mpmath import mp, j1, taylor, expm1
    # The j1 (Bessel J_1) function from mpmath will compute with mp.dps precision
    # The Taylor series computation will also respect mp.dps
    poly = taylor(lambda val: j1(val), 0, 46)
    # print(poly)
    real_i = 0
    print_double_double("a0: ", DD(poly[1]))
    print_double_double("a1: ", DD(poly[3]))
    print_double_double("a2: ", DD(poly[5]))
    print_double_double("a3: ", DD(poly[7]))
    print_double_double("a4: ", DD(poly[9]))
    print("series: [")
    for i in range(11, 46, 2):
        print(f"{double_to_hex(poly[i])},")
        real_i = real_i + 1
    print("],")

    print("};")

    print(f"poly {poly}")

print_expansion_at_0()

print(f"static J1_COEFFS: [TaylorExtendedSeries; {len(j1_zeros) + 1}] = [")

for i in range(0, len(j1_zeros)):
    k_range = j1_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    f_poly = bessel_J(1, x)
    x0 = mp.mpf(k_range)
    from mpmath import mp, j1, taylor, expm1
    # The j1 (Bessel J_1) function from mpmath will compute with mp.dps precision
    # The Taylor series computation will also respect mp.dps
    poly = taylor(lambda val: j1(val), x0, 23)
    # print(poly)
    print_taylor_coeffs(poly, CDR(k_range))
    prev_zero = j1_zeros[i]

print("];")


Zero 1: x ≈ 3.8317059702075111360386472370009869
Zero 2: x ≈ 7.0155866698156179595002868154551834
Zero 3: x ≈ 10.173468135063002293350109539460391
Zero 4: x ≈ 13.323691936314220995996038254816085
Zero 5: x ≈ 16.470630050877630878858326468616724
Zero 6: x ≈ 19.615858510468239472857021610252559
Zero 7: x ≈ 22.760084380592772390627942513674498
Zero 8: x ≈ 25.903672087618382136042782803997397
Zero 9: x ≈ 29.046828534916855346637021284550428
Zero 10: x ≈ 32.189679910974405174783896654844284
Zero 11: x ≈ 35.332307550083875469226768473163247
Zero 12: x ≈ 38.474766234771614392684568883851171
Zero 13: x ≈ 41.617094212814450315818248782306910
Zero 14: x ≈ 44.759318997652826510602608323097229
Zero 15: x ≈ 47.901460887185457693249190924689174
Zero 16: x ≈ 51.043535183571506763655634131282568
Zero 17: x ≈ 54.185553641061318330685026012361050
Zero 18: x ≈ 57.327525437901009297547716414555907
Zero 19: x ≈ 60.469457845347498903265659464523196
Zero 20: x ≈ 63.611356698481237970099755330011249
Zero 21: 

In [4]:
def print_expansion_at_0_f():
    print(f"pub(crate) const J1_MACLAURIN_SERIES: [u64; 13] = [")
    from mpmath import mp, j1, taylor, expm1
    mp.prec = 53
    poly = taylor(lambda val: j1(val), 0, 46)
    # print(poly)
    real_i = 0
    z = 0
    for i in range(1, 46, 2):
        print(f"{double_to_hex(poly[i])},")
        real_i = real_i + 1
    print("];")

    print(f"poly {poly}")

print_expansion_at_0_f()

pub(crate) static J1_MACLAURIN_SERIES: [u64; 13] = [
0x3fe0000000000000,
0xbfb0000000000000,
0x3f65555555555555,
0xbf0c71c71c71c71c,
0x3ea6c16c16c16c17,
0xbe3845c8a0ce5129,
0x3dc27e4fb7789f5c,
0xbd4522a43f65486a,
0x3cc2c9758daf5cd0,
0xbc3ab81ea75fcdf4,
0x3baf17697cf1cf13,
0xbb1e2637bef9ff1a,
0x3a88bce58901a35e,
0xb9f165e7c2d153f3,
0x39553585cdcbfb10,
0xb8b69f7da8510bcd,
0x38154ad09e6a6576,
0xb771d028acb00491,
0x36caaae78f4066a6,
0xb621f72d8389b3a3,
0x3575e69de22df5ce,
0xb4c84564b82a1184,
0x34188f11edf3ed4c,
];
poly [mpf('0.0'), mpf('0.5'), mpf('0.0'), mpf('-0.0625'), mpf('0.0'), mpf('0.0026041666666666665'), mpf('0.0'), mpf('-5.4253472222222219e-5'), mpf('0.0'), mpf('6.781684027777778e-7'), mpf('0.0'), mpf('-5.6514033564814812e-9'), mpf('0.0'), mpf('3.3639305693342149e-11'), mpf('0.0'), mpf('-1.5017547184527747e-13'), mpf('0.0'), mpf('5.2144261057388011e-16'), mpf('0.0'), mpf('-1.4484516960385557e-18'), mpf('0.0'), mpf('3.2919356728148996e-21'), mpf('0.0'), mpf('-6.2347266530585217e-24

In [10]:
# Taylor series for f32
mp.prec = 53
print(f"pub(crate) static J1F_COEFFS: [[u64; 13]; {len(j1_zeros)}] = [")

def get_constant_term(poly, y):
    for term in poly.operands():
        if term.is_constant():
            return term

def print_taylor_coeffsf(poly, x0):
    print("[")
    for i in range(0, 13):
        coeff = poly[i]
        print(f"{double_to_hex(coeff)},")
    print("],")

prev_zero = 0

for i in range(0, len(j1_zeros)):
    k_range = j1_zeros[i]
    range_diff = k_range - prev_zero
    g_c = 1

    f_poly = bessel_J(1, x)
    x0 = mp.mpf(k_range)
    from mpmath import mp, j1, taylor, expm1
    # The j1 (Bessel J_1) function from mpmath will compute with mp.dps precision
    # The Taylor series computation will also respect mp.dps
    poly = taylor(lambda val: j1(val), x0, 23)
    # print(poly)
    print_taylor_coeffsf(poly, CDR(k_range))
    prev_zero = j1_zeros[i]

print("];")

pub(crate) static J1F_COEFFS: [[u64; 13]; 51] = [
[
0x3fe29ea3d19f035f,
0xbd1905281527b4c2,
0xbfca41115c5df220,
0x3f78d1448e6ffac0,
0x3f8c441a2f9de1f1,
0xbf386671c18b11ca,
0xbf39e2504ddc75c7,
0x3ee34ccbca0c7c14,
0x3eda4973784d103e,
0xbe81045322aab053,
0xbe70fae0da6cdcbc,
0x3e13546cef5ed578,
0x3dfe5ee82e6676a8,
],
[
0x0000000000000000,
0xbfd9c6cf582cbf81,
0x3faae8a39f51ace8,
0x3fab589d1da13908,
0xbf7537544c331d97,
0xbf624b3409959066,
0x3f26e4c2d5354217,
0x3f083a06e30c410e,
0xbec9799d4c9f253b,
0xbea33825cd2e2c1a,
0x3e617069233e9163,
0x3e34569b22afc3cc,
0xbdf03b9e96510562,
],
[
0xbfd626ee83500bf2,
0xbd30b63e708daa6a,
0x3fc55f6bec9efa1c,
0xbf83d23336fcfd84,
0xbf88c77a983a0968,
0x3f45cdc98db1be46,
0x3f373576ff46efcb,
0xbef2461447d7aadd,
0xbed7b853456b707f,
0x3e90abfc6827431b,
0x3e6ea7a1ee2614e8,
0xbe235c0413e00c13,
0xbdfb5c5d512fbd7b,
],
[
0x0000000000000000,
0x3fd33518b3874e8b,
0xbf95e70dc603629f,
0xbfa80c83bdeee5b1,
0x3f69a4b292e3de2f,
0x3f613fbc7d698219,
0xbf207358bbdbff88,
0xbf0796a751f